# Model evaluation

## Goal

Has the model learned to draw?

## Imports

In [ ]:
from tqdm.auto import tqdm
from vllm import LLM, SamplingParams
from vllm.lora.request import LoRARequest
from transformers import AutoTokenizer, AutoConfig
import matplotlib.pyplot as plt
import matplotlib as mpl

from arc25.training_tasks import *
from arc25.encoders import create_grid_encoder
from arc25.prompting import create_prompt_from_task, pretty_print_prompt
from arc25.plot import plot_task
from arc25.code_execution import safe_code_execution
from arc25.utils import set_random_seed

plt.plot()
plt.close('all')
plt.rcParams["figure.figsize"] = (20, 5)
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['font.size'] = 12

## Load model

In [ ]:
base_model_path = '/home/gbarbadillo/models/Qwen2.5-Coder-0.5B-Instruct'
base_model_path = '/mnt/hdd0/Kaggle/arc25/trainings/20250430_first_trainings/steps_6400/model-6400'
# lora_path = '/mnt/hdd0/Kaggle/arc25/trainings/20250430_first_trainings/steps_6400/checkpoint-6400'
# lora_path = '/mnt/hdd0/Kaggle/arc25/trainings/20250430_first_trainings/random_seed_5_no_dora/checkpoint-200'
lora_path = '/mnt/hdd0/Kaggle/arc25/trainings/20250430_first_trainings/random_seed_4_no_dora_rank16/checkpoint-50'

In [ ]:
llm = LLM(
    model=base_model_path,
    enable_lora=True,
    trust_remote_code=True,
    dtype='auto',
    tensor_parallel_size=1, # to use 2 gpus
    max_model_len=10240,
    disable_log_stats=True,
    max_num_seqs=255, # default is supposed to be 256 I have used it to solve some weird illegal memory error
    enforce_eager=True,
)

In [ ]:
lora_request = LoRARequest(lora_name='lora', lora_int_id=1, lora_path=lora_path)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(lora_path)

## Inference

In [ ]:
task_generator = RandomDrawingTaskOnEmptyImg()
prompt_version = 'code-from-examples-v3'
grid_encoder = create_grid_encoder('GridShapeEncoder(RowNumberEncoder(MinimalGridEncoder()))')

In [ ]:
def evaluate_model(n_tasks, task_generator, sampling_params, random_seed=42):
    set_random_seed(random_seed)
    tasks = [task_generator.sample() for _ in range(n_tasks)]
    prompts = [
        create_prompt_from_task(
            task, prompt_version=prompt_version, grid_encoder=grid_encoder, tokenizer=tokenizer, is_train_prompt=False)
        for task in tasks
    ]
    request_output = llm.generate(prompts, sampling_params, use_tqdm=True)
    predicted_codes = [[output.text.replace('\n```', '') for output in task_output.outputs] for task_output in request_output]

    pass_n, accuracy, mean_correct_pixels, max_correct_pixels = [], [], [], []
    for task, task_predicted_codes in tqdm(zip(tasks, predicted_codes), total=len(tasks), desc='evaluating'):
        predicted_outputs = [safe_code_execution(predicted_code, task.inputs) for predicted_code in task_predicted_codes]
        pass_n.append(np.mean([any(np.all(output == predicted_output[idx]) for predicted_output in predicted_outputs)for idx, output in enumerate(task.outputs)]))
        accuracy.append(np.mean([np.mean([np.all(output == predicted_output[idx]) for predicted_output in predicted_outputs]) for idx, output in enumerate(task.outputs)]))
        mean_correct_pixels.append(np.mean([np.mean([np.mean(output == predicted_output[idx]) for predicted_output in predicted_outputs]) for idx, output in enumerate(task.outputs)]))
        max_correct_pixels.append(np.mean([np.max([np.mean(output == predicted_output[idx]) for predicted_output in predicted_outputs]) for idx, output in enumerate(task.outputs)]))
    metrics = {
        f'acc@{sampling_params.n}': np.mean(accuracy),
        f'pass@{sampling_params.n}': np.mean(pass_n),
        f'mean_correct_pixels': np.mean(mean_correct_pixels),
        f'max_correct_pixels': np.mean(max_correct_pixels),
    }
    metrics = {key: float(value) for key, value in metrics.items()}
    for key, value in metrics.items():
        print(f'{key}: {value:.2%}')
    return metrics

In [ ]:
n_tasks = 256
metrics = []
temperatures = [0.1, 0.2, 0.3, 0.4, 0.5]
for temperature in temperatures:
    sampling_params = SamplingParams(n=8, temperature=temperature, max_tokens=1024, logprobs=0, skip_special_tokens=False)
    print(f"Evaluating with temperature {temperature}")
    metrics.append(evaluate_model(n_tasks=n_tasks, task_generator=task_generator, sampling_params=sampling_params))

In [ ]:
for plot_idx, key in enumerate(metrics[0], 1):
    plt.subplot(1, len(metrics[0]), plot_idx)
    plt.title(key)
    plt.plot(temperatures, [metric[key] for metric in metrics], marker='o')
    plt.xlabel('temperature')
    plt.ylabel(key)
    plt.grid()
plt.suptitle('Effect of the temperature on the metrics')
plt.tight_layout()

In [ ]:
raise

## Manual created tasks

In [ ]:
#outputs = llm.generate([prompt], sampling_params, use_tqdm=True, lora_request=lora_request)

In [ ]:
input_img = create_img((10, 10), color=0)
output_img = draw_rectangle(input_img.copy(), (0, 0), (4, 4), color=1)
output_img = draw_rectangle(output_img, (5, 5), (9, 9), color=1)

task = Task(inputs=[input_img], outputs=[output_img], code='', name='manual')
plot_task(task)
prompt = create_prompt_from_task(
    task, prompt_version=prompt_version, grid_encoder=grid_encoder, tokenizer=tokenizer, is_train_prompt=False)
outputs = llm.generate([prompt], sampling_params, use_tqdm=True)
print(outputs[0].outputs[0].text)

In [ ]:
input_img = create_img((10, 10), color=0)
output_img = input_img.copy()
for x in range(0, input_img.shape[1], 2):
    draw_vertical_line(output_img, x, color=x)

task = Task(inputs=[input_img], outputs=[output_img], code='', name='manual')
plot_task(task)
prompt = create_prompt_from_task(
    task, prompt_version=prompt_version, grid_encoder=grid_encoder, tokenizer=tokenizer, is_train_prompt=False)
outputs = llm.generate([prompt], sampling_params, use_tqdm=True)
print(outputs[0].outputs[0].text)

In [ ]:
input_img = create_img((10, 10), color=0)
output_img = input_img.copy()
for x in range(0, input_img.shape[1], 1):
    draw_vertical_line(output_img, x, color=x)

task = Task(inputs=[input_img], outputs=[output_img], code='', name='manual')
plot_task(task)
prompt = create_prompt_from_task(
    task, prompt_version=prompt_version, grid_encoder=grid_encoder, tokenizer=tokenizer, is_train_prompt=False)
outputs = llm.generate([prompt], sampling_params, use_tqdm=True)
print(outputs[0].outputs[0].text)

In [ ]:
input_img = create_img((10, 10), color=0)
draw_horizontal_line(input_img, 4, color=1)
output_img = input_img.copy()
for x in range(0, input_img.shape[1], 2):
    draw_vertical_line(output_img, x, color=x)

task = Task(inputs=[input_img], outputs=[output_img], code='', name='manual')
plot_task(task)
prompt = create_prompt_from_task(
    task, prompt_version=prompt_version, grid_encoder=grid_encoder, tokenizer=tokenizer, is_train_prompt=False)
outputs = llm.generate([prompt], sampling_params, use_tqdm=True)
print(outputs[0].outputs[0].text)

- It is limited by the number of drawing functions in the train set
- It has only been trained with blank images, that does not require a good comparison between the images.

TODO:

- I want to visualize the transformation of the code.
- Also compute some metrics of accuracy